<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/python_/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. ResNet50 모델 로드:

ResNet50 모델은 사전 학습된 가중치(weights='imagenet')로 로드됩니다. 이 모델은 ImageNet 데이터셋에서 학습된 모델로, 1000개의 클래스를 예측할 수 있습니다.

2. 이미지 로드:

sklearn에서 제공하는 샘플 이미지를 사용합니다. load_sample_image('china.jpg') 또는 load_sample_image('flower.jpg')로 이미지를 로드할 수 있습니다.

3. 이미지 전처리:

이미지를 ResNet50 모델에 맞는 입력 크기인 224x224로 조정합니다. 이때 이미지 배열을 numpy.resize를 사용해 조정합니다.
전처리 함수 preprocess_input을 사용하여 이미지 데이터를 ResNet50이 기대하는 형식으로 변환합니다. 이 함수는 픽셀 값을 적절한 범위로 조정합니다.

4. 예측 수행:

model.predict(image)를 사용하여 예측을 수행합니다. 이 모델은 입력된 이미지에 대한 예측 확률을 반환합니다.

5. 예측 결과 디코딩:

decode_predictions를 사용하여 예측 결과를 해석 가능한 레이블로 디코딩합니다. top=3을 사용하여 가장 가능성이 높은 3개의 클래스를 출력합니다.

6. 결과 출력:

각 예측 결과에 대해 레이블과 해당 확률을 출력합니다.

In [1]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# 1. ResNet50 모델 로드
model = ResNet50(weights='imagenet')

# 2. sklearn에서 제공하는 샘플 이미지 로드
# 사용 가능한 샘플 이미지: 'china.jpg', 'flower.jpg'
image = load_sample_image('china.jpg')  # 또는 'flower.jpg'
image = img_to_array(image)

# 3. 이미지 전처리
# ResNet50 모델에 입력할 수 있도록 이미지 크기를 224x224로 조정하고, 전처리
image = np.resize(image, (224, 224, 3))
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 4. 예측 수행
predictions = model.predict(image)

# 5. 예측 결과 디코딩 및 출력
decoded_predictions = decode_predictions(predictions, top=3)[0]
print("Predicted:", decoded_predictions)

# 출력된 결과를 해석
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i+1}: {label} ({score:.2f})")


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: [('n04152593', 'screen', 0.10869243), ('n02840245', 'binder', 0.104571015), ('n06359193', 'web_site', 0.049777426)]
1: screen (0.11)
2: binder (0.10)
3: web_site (0.05)


* screen (0.11):

  모델이 이미지에서 "스크린(screen)"이라고 예측한 확률이 약 11%입니다.
이 예측이 가장 높은 확률로, 모델은 이미지가 스크린과 관련이 있을 가능성이 있다고 판단했습니다.

* binder (0.10):

  모델은 또한 이미지가 "바인더(binder)"일 가능성을 약 10%로 예측했습니다.
이는 바인더(서류철)와 관련된 물체를 인식한 것으로 보입니다.

* web_site (0.05):

  세 번째로, 모델은 이미지가 "웹 사이트(web site)"와 관련이 있을 가능성을 약 5%로 예측했습니다.
  이 결과는 이미지에 웹 페이지와 관련된 요소가 있을 수 있다고 인식한 결과입니다.

In [2]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# 1. ResNet50 모델 로드 및 미세 조정 설정
base_model = ResNet50(weights='imagenet', include_top=False)  # 최상위 레이어 제거

# 2. 레이어 추가 및 모델 확장
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 추가
predictions = Dense(1000, activation='softmax')(x)  # ImageNet 데이터셋의 클래스 수

# 새 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 3. 사전 학습된 레이어 동결
for layer in base_model.layers:
    layer.trainable = False  # ResNet50의 기존 레이어는 학습되지 않도록 동결

# 4. 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 5. 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 6. 샘플 이미지 로드 및 전처리
image = load_sample_image('china.jpg')  # 예제 이미지
image = img_to_array(image)
image = np.resize(image, (224, 224, 3))  # ResNet50 입력 크기에 맞추기
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 7. 예측 수행
y_pred = model.predict(image)

# 8. 예측 결과 디코딩 및 출력
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)

# 9. 모델 학습 (이 부분은 예제에서 생략됩니다)
# 일반적으로 대규모 데이터셋을 사용하여 학습을 진행해야 합니다.
# datagen.flow(...) 또는 model.fit(...)을 사용하여 데이터 증강된 이미지로 학습 진행 가능

# 조기 종료 설정
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 실제 데이터로 모델 학습
# model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[early_stopping])



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Predicted: [('n02782093', 'balloon', 0.0087746605), ('n03697007', 'lumbermill', 0.00767095), ('n03478589', 'half_track', 0.0074001565)]


* balloon (0.0088):
모델은 이미지의 일부를 "balloon" (풍선)이라고 인식했을 가능성을 약 0.88%로 예측했습니다.
* lumbermill (0.0077):
모델은 이미지의 일부를 "lumbermill" (제재소)라고 인식할 가능성을 약 0.77%로 예측했습니다.
* half_track (0.0074):
모델은 이미지의 일부를 "half_track" (반트랙, 장갑차의 일종)으로 인식할 가능성을 약 0.74%로 예측했습니다.